In [ ]:
#splitting every slash into new index of sentence
a_list = a.apply(lambda x : x.str.split('/'))

for i in a_list:
    tmp = pd.DataFrame.from_records(a_list[i].tolist()).stack().reset_index(level=1, drop=True).rename('new_{}'.format(i))
    a = a.drop(i, axis=1).join(tmp)

a = a.reset_index(drop=True)

In [ ]:
import pandas as pd
data=pd.read_excel(r"C:\Users\Dell\Desktop\r&d\sponttest.xlsx")

In [ ]:
data['content1']=data['content']

In [ ]:
data.head()

In [ ]:
data['content']=[x for x in data['content'].str.lower()]

In [ ]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
import nltk
import re
from pywsd.utils import lemmatize_sentence
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [ ]:
data['content'] = data['content'].str.replace("/", " and")

In [ ]:
data['content']= data['content'].str.replace("[^a-zA-Z#]", " ")
data.head()

In [ ]:
#Sentiment part
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
#nltk vader lexicon 
final_sentimentt=[]
for i in data['content'].values:
    try:
        senti=SentimentIntensityAnalyzer()
        analysis=senti.polarity_scores(i)
        final_sentimentt.append(analysis['compound'])
    except:
        final_sentimentt.append(0)
data['final_sentimentt']=final_sentimentt

pos=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt>0]
neg=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt<0]
hpos=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt>0.75]
hneg=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt<-0.25]

#Converting the polarity values from continuous to categorical
data['final_sentimentt'][data.final_sentimentt==0]= 0
data['final_sentimentt'][data.final_sentimentt > 0]= 1
data['final_sentimentt'][data.final_sentimentt< 0]= -1

In [ ]:
#changing to needed format(senti)
value = {1.0:'pos',-1.0:'neg',0.0:'neu'} 
data.final_sentimentt = [value[item] for item in data.final_sentimentt]  

In [ ]:
places=['dl']
#xtracting loc and location 
pattern='|'.join(places)
x=data['content'].str.contains(pattern)
data['Sentiment']=x.replace((True,False),('negative','positive'))

In [ ]:
data['content'] = data['content'].str.replace("dl", " ")

In [ ]:
#Aspect extraction part
!pip install rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
from rake_nltk import Rake
from nltk.corpus import stopwords 
from rake_nltk import Rake
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
import nltk
import re
from pywsd.utils import lemmatize_sentence
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [ ]:
#extracting aspect with higher ranking phrases (top5 min)
aspect_adj_string =[]
for i in data['content'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=2)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:10]
        aspect_adj_string.append(b)
    except:
        aspect_adj_string.append(0)
data['aspect_adj_string']=aspect_adj_string

In [ ]:
#topic modelling
productquality=['dandruff','hairfall','hair','reduced','dont','buy','increased','conditioner','conditionerror','shiny','soft',
                'doesnt','support','wash','scalp','sticky','washing','scalps','oily','dirty','grow','hairs','control','washes',
                'smooth','silky','reduces','fall','regrowing','growing','problem','healthy','stopped','variant','decreasing',
                'wash','mild','fall','moisturizing','using','conditioner','moisturizer','smoothens','frizzy','hair','manageable',
                'gradually','improved','type','results','promising','promise','use','life','exotic','lathers','loss','straight',
               'delicate','utilizing','utilize','sleek','dry','tangle','free','satisfied','hairwash','softens','soften','thick',
               'rough','hairs','suits','decreased','reduced','completely','reduce','daily','palm','performed','manage','herbal',
               'parabeen','antihairfall','baldness','shine','real','feel','healthier','healthy','effects','cures','cure','really',
               'working','work','controll','control','stopped','stop','consistency','cooling','refreshing','refresh','slightly',
               'last','years','difference','length','doesnt','literally','paraben','natural','protection','protect','relief','sulfate',
               'sulphate','improve','texture','feels','feel','quality','expected','product','ayurvedic','shampoos','chemicals','chemical','himalaya','patanjali','nyle','chik','dubious',
        'like','shampoo','smelling','bad','good','value','last','ok','never','buy','poor','waste','normal','purchase','didnt','average',
        'improvement','nothing','long','time','fake','worthful','worth','almost','gone','setisfied','satisfied','osm','name','opposite',
         'products','gud','gd','scent','brand','manufacturer','indian','love','worth','experience','nice','works','tried','disappointed',
        'fragrance','best','deal','liked','always','use','super','great','necharal','love','grt','cosmetic','amazing','branded',
        'effective','smell','beautiful','thanks','wonderful','assured','fantastic','favorite','excellent','fine','happy','awesome',
        'super','fav','useful','recommended','recommended','ok']

In [ ]:
packaging=['bottle','damaged','pack','leaked','package','packing','sealed','parcel','slippery','wet','cellophane','tap','tape','packaging',
          'plaster','piece','tightly','fix','cap','leaking','wasted','leakage','condition','opening','covered','lids','approx','ml','seal',
          'damage','open','spilling','lid','defective','spoiled','packet','opened','delivery','packing']

pricing=['pricing','price','pric','money','rs','rupees','inr','paisa','cheap','worth','charged','pay','mrp','cost']


In [ ]:
lemmatizer = WordNetLemmatizer()
data['content'] = data['content'].apply(lemmatize_sentence)

In [ ]:
def matcher(query):
    s=0
    it=0
    wl=0
         

    for x in query: 
        if x in productquality:        
            s +=1                    
        if x in packaging:
            it +=1                
        if x in pricing:
            wl +=1
        
    max_count=max(s,it,wl)
    #print(max_count)
    if max_count==s:
        return 'productquality'
    elif max_count==it:
        return 'packaging'
    elif max_count==wl:
        return 'pricing'
data['class1']=data['content'].apply(matcher)

In [ ]:
def matcher(query):
    s=0
    it=0
    wl=0
         

    for x in query: 
        if x in productquality:        
            s +=1                    
        if x in packaging:
            it +=1                
        if x in pricing:
            wl +=1
        
    max_count=max(s,it,wl)
    max_count1=max_count-1
    if max_count1==s:
        return 'productquality'
    elif max_count1==it:
        return 'packaging'
    elif max_count1==wl:
        return 'pricing'
data['class2']=data['content'].apply(matcher)

In [ ]:

data.to_excel(r"C:\Users\Dell\Desktop\r&d\sponttest0.xlsx")